In [1]:
import torch
import torchvision
from transformers import AutoTokenizer
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(0)
from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

/home/dchenbs/anaconda3/envs/seq2seq-ae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def segment_one_image(segmenter, image_path, visualize=False):

    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)
    segment_masks = segmenter(image_path)

    def visualized_masks(masks, image):
        canvas = np.ones_like(image) * 255
        masks = sorted(masks, key=lambda x: x['area'], reverse=True)
        for mask in masks:
            average_color = np.mean(image[mask['segmentation'] == 1], axis=0)
            canvas[mask['segmentation'] == 1] = average_color

            # visualize segment boundary
            contours, _ = cv2.findContours(mask['segmentation'].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(canvas, contours, -1, (200, 200, 200), 1)
        return canvas
    
    if visualize:
        plt.figure(figsize=(20, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(image_np)
        plt.axis('off')

        canvas = visualized_masks(segment_masks, image_np)
        plt.subplot(1, 2, 2)
        plt.imshow(canvas)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    for segment_mask in segment_masks:
        segment_mask['image_index'] = 0
        segment_mask['segmentation'] = torch.from_numpy(segment_mask['segmentation'])
        
    return segment_masks, [image]

In [3]:
# from model.modeling_multimodal_phi2 import *
# model = PhiForMultimodal.from_pretrained(
#     "microsoft/phi-2",
#     w_segment_loss=1.0,
#     w_bbox_loss=1.0,
#     ).to(DEVICE)

# model.load_seqae('/home/dchenbs/workspace/Seq2Seq-AutoEncoder/runs/Nov28_20-50-04_host19-SA1B-[327MB-16queries-1024]-[lr1e-05-bs16x1step-8gpu]/checkpoints/checkpoint_ep2_step3200k')

# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
# tokenizer.add_tokens(["<|startofimage|>", "<|endofimage|>", "<|seg|>"])
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# model.resize_token_embeddings(len(tokenizer))
# model.special_token_id_mappinmg = {
#     "<|startofimage|>": tokenizer.convert_tokens_to_ids("<|startofimage|>"),
#     "<|endofimage|>": tokenizer.convert_tokens_to_ids("<|endofimage|>"),
#     "<|seg|>": tokenizer.convert_tokens_to_ids("<|seg|>"),
#     "<|endoftext|>": tokenizer.convert_tokens_to_ids("<|endoftext|>"),
#     "[PAD]": tokenizer.convert_tokens_to_ids("[PAD]"),
# }


In [4]:
import sys
sys.path.append("segmentation")

from segmentation import Segmenter
segmenter = Segmenter('mobile_sam_v2', '/home/dchenbs/workspace/cache/sam_weights/mobile_sam_v2/l2.pt')

/home/dchenbs/workspace/Seq2Seq-AutoEncoder/segmentation/tinyvit/tiny_vit.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with tinyvit.tiny_vit.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/dchenbs/workspace/Seq2Seq-AutoEncoder/segmentation/tinyvit/tiny_vit.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with tinyvit.tiny_vit.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/dchenbs/workspace/Seq2Seq-AutoEncoder/segmentation/tinyvit/tiny_vit.py:656: UserWarning: Overwriting tiny_vit_21m_224 in registry with tinyvit.tiny_vit.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/dchenbs/workspace/Seq

checkpoint_load_scucess


In [21]:
from data.sharegpt4v import ShareGPT4V
sharegpt4v_path = '/home/dchenbs/workspace/datasets/sharegpt4v/ShareGPT4V'

# annotation_file = os.path.join(sharegpt4v_path, 'share-captioner_coco_lcs_sam_1246k_1107.json')
# annotation_file = os.path.join(sharegpt4v_path, 'sharegpt4v_instruct_gpt4-vision_cap100k.json')
annotation_file = os.path.join(sharegpt4v_path, 'sharegpt4v_mix665k_cap23k_coco-ap9k_lcs3k_sam9k_div2k.json')


dir_mapping = {
    'sam/images': '/home/dchenbs/workspace/datasets/sa1b',
    'coco/train2017': '/home/dchenbs/workspace/datasets/coco2017/images/train2017',
    'llava/llava_pretrain/images': '/home/dchenbs/workspace/datasets/sharegpt4v/LLaVA-Pretrain',
    'wikiart/images': '/home/dchenbs/workspace/datasets/sharegpt4v/WebData/wikiart/images',
    'share_textvqa/images': '/home/dchenbs/workspace/datasets/sharegpt4v/WebData/share_textvqa/images',
    'web-celebrity/images': '/home/dchenbs/workspace/datasets/sharegpt4v/WebData/web-celebrity/images',
        'Choi_Min-sik': 'Choi_Min_sik', 
        'Lee_Byung-hun': 'Lee_Byung_hun',
    'web-landmark/images': '/home/dchenbs/workspace/datasets/sharegpt4v/WebData/web-landmark/images',
    'vg/VG_100K': '/home/dchenbs/workspace/datasets/VisualGenome/VG_100K',
    'vg/VG_100K_2': '/home/dchenbs/workspace/datasets/VisualGenome/VG_100K_2',
    'gqa/images': '/home/dchenbs/workspace/datasets/gqa/images',
    'ocr_vqa/images': '/home/dchenbs/workspace/datasets/sharegpt4v/ocr_vqa/images',
    'textvqa/train_images': '/home/dchenbs/workspace/datasets/sharegpt4v/textvqa/train_images',
}

dataset = ShareGPT4V(annotation_file, dir_mapping)
failed_samples = dataset.validate_exist()

Total samples: 665058, after removing text-only samples: 624370


100%|██████████| 624370/624370 [00:01<00:00, 314744.34it/s]

Found 355 samples failed to load.


In [ ]:

img_dir = '/home/dchenbs/workspace/datasets/coco2017/images/val2017'
img_path = os.path.join(img_dir, random.choice(os.listdir(img_dir)))
segment_masks, images = segment_one_image(segmenter, img_path, visualize=False)

In [ ]:
inputs = tokenizer([
    'Hellow world!',
    'This is a segment token: <|startofimage|><|seg|><|endofimage|><|endoftext|>',
    'This is an image of a cat with 4 tokens: <|startofimage|><|seg|><|seg|><|seg|><|seg|><|endofimage|>!<|endoftext|>',
    ], return_tensors="pt", return_attention_mask=False, padding=True).to(DEVICE)

print(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=False))
print(tokenizer.decode(inputs['input_ids'][1], skip_special_tokens=False))
print(tokenizer.decode(inputs['input_ids'][2], skip_special_tokens=False))

print(inputs, inputs['input_ids'].shape)

print(model.special_token_id_mappinmg)
print('\n\n\n')
output = model(
    input_ids=inputs['input_ids'],
    segment_masks=[[], segment_masks[:1], segment_masks[:4]],
    images=images[:4],
    labels=inputs['input_ids'],
    )

# outputs = model.generate(**inputs, max_length=200)
# text = tokenizer.batch_decode(outputs)[0]
# print(text)



In [ ]:
print(output.keys())